# CVNNによる非線形歪補償
複素数を入力とする3層ANNによる補償

In [2]:
#import
import sys
import os
import time
import datetime
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

sys.path.append('../')
from pyopt.util import save_pickle, load_pickle

# 1. Preprocessing

## 1.1 データの整形

In [3]:
def data_shaping(input_signal, signal, sampling, tap, max_tap, n):
    """
    input_signal: 伝送前の信号
    signal: 伝送後の信号
    max_tap: 最大の同時入力シンボル数
    tap: 同時入力シンボル数
    
    signal = [x_0, x_1, ... , x_(n-1)]
      |
      |
      v
    x = [[x_0, x_1, ... , x_tap-1],
            [x_1, x_2, ..., x_tap],
                   .
                   .
                   .
            [x_(n-tap), x_(n-tap+1), ..., x(n-1)]]
      |
      |
      v
    x = [[i_0, q_0, i_1, q_1, ... , i_(tap-1), q_(tap-1)],
            [i_1, q_1, i_2, q_2, ... , i_tap, q_tap],
                   .
                   .
                   .
            [i_(n-tap), q_(n-tap), i_(n-tap+1), q_(n-tap+1), ..., i_(n-1), q_(n-1)]] (batch, input_dim) input_dim = tap * 2
    
    y  (batch, output_dim) output_dim = 2
    """
    
    x = np.zeros((len(input_signal) // n - (max_tap - 1), sampling * tap, 2), dtype=float)
    y = np.zeros((len(input_signal) // n - (max_tap - 1), 2), dtype=float)
    if sampling == 1:
        for i, center in enumerate(range(max_tap // 2, len(input_signal) // n - max_tap // 2)):
            for j, symbol in enumerate(range(tap)):
                x[i, j, 0] = signal[n * (center - tap // 2 + j) + n // 2].real
                x[i, j, 1] = signal[n * (center - tap // 2 + j) + n // 2].imag
            y[i, 0] = input_signal[n // 2 + n * center].real
            y[i, 1] = input_signal[n // 2 + n * center].imag
    else:
        for i, center in enumerate(range(max_tap // 2, len(input_signal) // n - max_tap // 2)):
            for j, symbol in enumerate(range(tap)):
                for k, sample in enumerate([round(_ * n / sampling) for _ in range(sampling)]):
                    x[i, j * sampling + k, 0] = signal[n * (center - tap // 2 + j) + sample].real
                    x[i, j * sampling + k, 1] = signal[n * (center - tap // 2 + j) + sample].imag
            y[i, 0] = input_signal[n // 2 + n * center].real
            y[i, 1] = input_signal[n // 2 + n * center].imag
    return x, y

In [6]:
#動作確認
sampling = 3
tap = 51
max_tap = 51

df_dir = '../data/input/prbs.csv'
df = pd.read_csv(df_dir, index_col=0)  # dataframe読み込み
condition = (df['N']==13) & (df['itr']==1) & (df['form']=='RZ16QAM') & (df['n']==32) & (df['equalize']==False) & (df['baudrate']==28) & (df['PdBm']==1)
sgnl = load_pickle(df[condition].iloc[0]['data_path'])  # dataframeから条件と合う行を取得し,pickleの保存先(data_path)にアクセス
lc = sgnl.linear_compensation(500, sgnl.signal['x_500'])
x, y = data_shaping(sgnl.signal['x_0'], lc, sampling, tap, max_tap, 32)  # ANNに入力できるようにデータを整形

print('x size: ', x.shape)
print('y size: ', y.shape)
print(x[0])
print(y[0])

x size:  (1998, 153, 2)
y size:  (1998, 2)
[[  9850.68513614 -21219.80262024]
 [ 13204.64011713 -32959.409542  ]
 [  6948.9465661  -23474.70595918]
 [  6572.22992075 -19764.76810906]
 [  4963.7554057  -28359.94776034]
 [  4484.76229895 -21853.46310266]
 [  6797.53679687 -25291.13803704]
 [ 10767.82650196 -34408.70724833]
 [  9678.50727356 -23175.84758057]
 [ 24185.99583942 -63906.15879268]
 [ 28765.59632506 -95588.70645972]
 [ 18657.13045424 -67387.64694757]
 [ 56175.5509019  -10325.27176417]
 [ 71641.59997349 -13264.43479735]
 [ 47605.46488907  -6410.10234191]
 [ 70448.73916762  22469.73798232]
 [ 92211.70544456  31290.2122476 ]
 [ 62700.35070749  20206.6647782 ]
 [-23472.17369949  -8746.64263679]
 [-29586.19186635 -10544.86139771]
 [-20101.04150411  -5371.96315595]
 [ 68619.96000997  22685.84254411]
 [ 93962.83654914  31040.0557247 ]
 [ 64874.50621444  18833.08566138]
 [-23270.56758099  69767.8172635 ]
 [-32415.55573636  97552.01834647]
 [-22512.10891961  70694.09360485]
 [ 66383.099

## 1.2 平均,標準偏差の計算

In [4]:
mean = np.mean(x)
std = np.std(x)

print('mean: ', mean)
print('std: ', std)

mean:  1053.2765739234778
std:  45950.91990246874


# 2. Dataset定義

In [4]:
class Dataset(data.Dataset):
    def __init__(self, x, y, mean, std):
        self.x, self.y, self.mean, self.std = x, y, mean, std
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        x = (x - self.mean) / self.std
        y = (y - self.mean) / self.std
        
        x = torch.Tensor(x)
        y = torch.Tensor(y)
        
        x_i = x[:, 0]
        x_q = x[:, 1]
        y_i = y[0]
        y_q = y[1]
        return x_i, x_q, y_i, y_q

In [8]:
#動作確認
train_dataset = Dataset(x=x, y=y, mean=mean, std=std)

index = 0
x_i, x_q, y_i, y_q = train_dataset.__getitem__(index)
x_array = x_i.detach().numpy()

print('mean: ', np.mean(x_array))
print('std: ', np.std(x_array))
print(x_i)
print(y_i)

mean:  -0.027470334
std:  0.95101625
tensor([ 0.1915,  0.2644,  0.1283,  0.1201,  0.0851,  0.0747,  0.1250,  0.2114,
         0.1877,  0.5034,  0.6031,  0.3831,  1.1996,  1.5362,  1.0131,  1.5102,
         1.9838,  1.3416, -0.5337, -0.6668, -0.4604,  1.4704,  2.0219,  1.3889,
        -0.5293, -0.7284, -0.5128,  1.4217,  2.0531,  1.4819,  0.2273,  0.2061,
         0.1058,  0.7858,  1.1000,  0.7800, -0.5345, -0.7225, -0.5002, -0.5536,
        -0.6890, -0.4607,  0.4913,  0.6844,  0.4074,  0.0098,  0.1566,  0.1687,
         0.4826,  0.6105,  0.4485, -1.3130, -1.7265, -1.1647, -1.2725, -1.6900,
        -1.1886,  1.0256,  1.4862,  1.0753,  0.5028,  0.7808,  0.4340,  0.1014,
         0.2612,  0.1293,  0.2132,  0.2080,  0.1040, -1.1466, -1.6223, -1.1647,
        -0.7815, -1.1080, -0.7573, -0.8030, -1.1768, -0.8036,  0.4602,  0.6795,
         0.4335, -0.3344, -0.3360, -0.2184, -0.7909, -1.0595, -0.7877, -0.1990,
        -0.2742, -0.2407, -0.0934, -0.2683, -0.3456, -0.7994, -1.1494, -0.8074,
   

In [9]:
batch_size = 100

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader}

# 3. Model定義

In [5]:
class CSigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x_i, x_q):
        x_i, x_q = x_i.clone(), x_q.clone()
        x_i, x_q = torch.sigmoid(x_i), torch.sigmoid(x_q)
        return x_i, x_q

In [7]:
class CReLU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x_i, x_q):
        x_i, x_q = x_i.clone(), x_q.clone()
        x_i, x_q = F.relu(x_i), F.relu(x_q)
        return x_i, x_q

In [8]:
class modReLU(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.in_features = in_features
        self.b = torch.tensor(-math.sqrt(1 / in_features), requires_grad=True)
    
    def forward(self, x_i, x_q):
        x_i, x_q = x_i.clone(), x_q.clone()
        norm = torch.sqrt(x_i ** 2 + x_q ** 2)
        
        out_i = F.relu(norm + self.b) * x_i / norm
        out_q = F.relu(norm + self.b) * x_q / norm
        return out_i, out_q

In [9]:
class ComplexLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # 重み定義 Xavierの初期値
        k = 1 / in_features
        weight_i = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_i = nn.Parameter(weight_i)
        weight_q = torch.empty(out_features, in_features).uniform_(-math.sqrt(k), math.sqrt(k))
        self.weight_q = nn.Parameter(weight_q)
        
        bias_i = torch.empty(out_features).uniform_(-k, k)
        self.bias_i = nn.Parameter(bias_i)
        bias_q = torch.empty(out_features).uniform_(-k, k)
        self.bias_q = nn.Parameter(bias_q)
        
    def forward(self, x_i, x_q):
        i = nn.functional.linear(x_i, self.weight_i) - nn.functional.linear(x_q, self.weight_q) + self.bias_i
        q = nn.functional.linear(x_i, self.weight_q) + nn.functional.linear(x_q, self.weight_i) + self.bias_q
        return i, q

In [10]:
class CVNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_neuron, activation):
        super().__init__()
        self.fc1 = ComplexLinear(input_dim, hidden_neuron)
        self.fc2 = ComplexLinear(hidden_neuron, output_dim)
        if activation == 'CSigmoid':
            self.activation = CSigmoid()
        elif activation == 'ZReLU':
            self.activation = ZReLU()
        elif activation == 'CReLU':
            self.activation = CReLU()
        elif activation == 'modReLU':
            self.activation = modReLU(input_dim)
    
    def forward(self, x_i, x_q):
        x_i, x_q = self.fc1(x_i, x_q)
        x_i, x_q = self.activation(x_i, x_q)
        x_i, x_q = self.fc2(x_i, x_q)
        return x_i, x_q

In [17]:
#動作確認
hidden_neuron = 300
activation = 'CSigmoid'
device = torch.device('cpu') # 'cuda' if torch.cuda.is_available() else 
print('Device available now:', device)

model = CVNN(input_dim=sampling*tap, output_dim=1, hidden_neuron=hidden_neuron, activation=activation).to(device)
for x_i, x_q, y_i, y_q in train_dataloader:
    out_i, out_q = model(x_i, x_q)
    print(out_i.shape)
    break

Device available now: cpu
torch.Size([100, 1])


# 4. train定義

In [11]:
def evm_score(x_i, x_q, y_i, y_q):
    tmp = 0
    for i in range(len(x_i)):
        tmp += ((x_i[i] - y_i[i]) ** 2 + (x_q[i] - y_q[i]) ** 2) / (y_i[i] ** 2 + y_q[i] ** 2)
    evm = torch.sqrt(tmp / len(x_i))
    return evm

In [12]:
class EVMLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x_i, x_q, y_i, y_q):
        return evm_score(x_i, x_q, y_i, y_q)

In [13]:
def train_model(device, model, dataloaders_dict, criterion, optimizer, epochs, epochs_section=None):
    for epoch in range(epochs):
        if epochs_section is not None:
            epoch += epochs_section[0]
            end_epoch = epochs_section[1]
        else:
            end_epoch = epochs
        
        start_time = time.time()
        
        for phase in dataloaders_dict.keys():
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            epoch_loss = 0.0
            epoch_evms = 0.0
            
            for x_i, x_q, y_i, y_q in dataloaders_dict[phase]:
                x_i, x_q = x_i.to(device), x_q.to(device)
                y_i, y_q = y_i.to(device), y_q.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    out_i, out_q = model(x_i, x_q)
                    loss = criterion(out_i, out_q, y_i, y_q)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss += loss.item() * x_i.size(0)
                    epoch_evms += (evm_score(out_i, out_q, y_i, y_q)) ** 2 * x_i.size(0)
            
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_evm = torch.sqrt(epoch_evms / len(dataloaders_dict[phase].dataset)) * 100
            
            duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
            print('{} | Epoch: {}/{} | {} Loss: {:.4} | EVM: {:.4}'.format(duration, epoch + 1, end_epoch, phase, epoch_loss, epoch_evm[0]))
    return model

In [22]:
#動作確認
epochs = 5
lr = 0.001

criterion = EVMLoss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs);

0:00:00 | Epoch: 1/5 | train Loss: 0.1415 | EVM: 14.35
0:00:00 | Epoch: 2/5 | train Loss: 0.1074 | EVM: 10.81
0:00:00 | Epoch: 3/5 | train Loss: 0.09289 | EVM: 9.328
0:00:00 | Epoch: 4/5 | train Loss: 0.09666 | EVM: 9.701
0:00:00 | Epoch: 5/5 | train Loss: 0.0901 | EVM: 9.05


# 5. 実行

In [14]:
def load_signal(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax):
    if signal_info['signal_type'] == 'prbs':
        return load_prbs(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax)
    elif signal_info['signal_type'] == 'random':
        return load_random(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax)
    elif signal_info['signal_type'] == 'image':
        return load_image(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax)

In [15]:
def load_prbs(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax):
    N = signal_info['N']
    itr = signal_info['itr']
    signal_condition = 'N=='+str(N)+'&itr=='+str(itr)
    signal_list = [N, itr] + [None] * 6
    
    # prbs.csvをpandasで読み込む
    t_df_dir = '../data/input/'
    t_df = pd.read_csv(t_df_dir + 'prbs.csv', index_col=0)
    
    # prbs.csvにおいて、指定した伝送条件を見たす行を抜き出す
    t_query = t_df.query(signal_condition + '&' + trans_condition)
    
    # if prbs.csvに指定した伝送条件がない or Lmax以外は満たすがLmaxだけ指定した条件未満: 何もしない if ある: 続ける
    if len(t_query) == 0 or t_query.iloc[0]['Lmax'] < Lmax:
        print('指定された伝送条件の信号が存在しません')
        return
    else:
        # 伝送信号を学習データに整形する
        sgnl = load_pickle(t_query.iloc[0]['data_path'])
        if lc:
            sgnl_distorted = sgnl.linear_compensation(Lmax, sgnl.signal['x_' + str(Lmax)])
        else:
            sgnl_distorted = sgnl.signal['x_' + str(Lmax)]
        x, y = data_shaping(sgnl.signal['x_0'], sgnl_distorted, sampling, tap, max_tap, n)
        return x, y, signal_condition, signal_list

In [16]:
def load_random(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax):
    seed = signal_info['seed']
    bit_num = signal_info['bit_num']
    signal_condition = 'seed=='+str(seed)+'&bit_num=='+str(bit_num)
    signal_list = [None] * 2 + [seed, bit_num] + [None] * 4
    
    # random.csvをpandasで読み込む
    t_df_dir = '../data/input/'
    t_df = pd.read_csv(t_df_dir + 'random.csv', index_col=0)
    
    # random.csvにおいて、指定した伝送条件を見たす行を抜き出す
    t_query = t_df.query(signal_condition + '&' + trans_condition)
    
    # if random.csvに指定した伝送条件がない or Lmax以外は満たすがLmaxだけ指定した条件未満: 何もしない if ある: 続ける
    if len(t_query) == 0 or t_query.iloc[0]['Lmax'] < Lmax:
        print('指定された伝送条件の信号が存在しません')
        return
    else:
        # 伝送信号を学習データに整形する
        sgnl = load_pickle(t_query.iloc[0]['data_path'])
        if lc:
            sgnl_distorted = sgnl.linear_compensation(Lmax, sgnl.signal['x_' + str(Lmax)])
        else:
            sgnl_distorted = sgnl.signal['x_' + str(Lmax)]
        x, y = data_shaping(sgnl.signal['x_0'], sgnl_distorted, sampling, tap, max_tap, n)
        return x, y, signal_condition, signal_list

In [17]:
def load_image(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax):
    target_dir = signal_info['target_dir']
    step = signal_info['step']
    image_number = signal_info['image_number']
    image_number_split = image_number.split(', ')
    ebtb = signal_info['ebtb']
    signal_condition = 'target_dir=="'+str(target_dir)+'"&step=='+str(step)+'&image_number=="'+image_number+'"&ebtb=='+str(ebtb)
    signal_list = [None] * 4 + [target_dir, step, image_number, ebtb]
    
    # image.csvをpandasで読み込む
    t_df_dir = '../data/input/'
    t_df = pd.read_csv(t_df_dir + 'image.csv', index_col=0)
    
    x = None
    y = None
    for i in range(len(image_number_split)):
        # image.csvにおいて、指定した伝送条件を見たす行を抜き出す
        t_query = t_df.query('target_dir=="'+str(target_dir)+'"&step=='+str(step)+'&image_number=='+image_number_split[i]+'&ebtb=='+str(ebtb) + '&' + trans_condition)
        
        # if image.csvに指定した伝送条件がない or Lmax以外は満たすがLmaxだけ指定した条件未満: 何もしない if ある: 続ける
        if len(t_query) == 0 or t_query.iloc[0]['Lmax'] < Lmax:
            print('指定された伝送条件の信号が存在しません')
            return
        else:
            # 伝送信号を学習データに整形する
            sgnl = load_pickle(t_query.iloc[0]['data_path'])
            if lc:
                sgnl_distorted = sgnl.linear_compensation(Lmax, sgnl.signal['x_' + str(Lmax)])
            else:
                sgnl_distorted = sgnl.signal['x_' + str(Lmax)]
            x_tmp, y_tmp = data_shaping(sgnl.signal['x_0'], sgnl_distorted, sampling, tap, max_tap, n)
            if x is None:
                x = x_tmp
                y = y_tmp
            else:
                x = np.concatenate([x, x_tmp])
                y = np.concatenate([y, y_tmp])
    return x, y, signal_condition, signal_list

In [18]:
def load_model(learn_condition, trans_condition, signal_condition, device, sampling, tap, neuron, epochs, activation, Lmax):
    # if CVNN.csv がある: pandasで読み込む if CVNN.csvがない: 新しいDataFrameを作る
    l_df_dir = '../data/params/CVNN.csv'
    if os.path.exists(l_df_dir):
        l_df = pd.read_csv(l_df_dir, index_col=0)
    else:
        cols = ['linear_compensation', 'sampling', 'tap', 'max_tap', 'batch_size', 'neuron', 'epochs', 'learning_rate', 'activation', 'N', 'itr', 'seed', 'bit_num', 'target_dir', 'step', 'image_number', 'ebtb', 'form', 'n', 'equalize', 'baudrate', 'PdBm', 'Ledfa', 'stepedfa', 'gamma', 'D', 'Alpha', 'NF', 'Lmax', 'ase', 'params_path', 'train_samples']
        l_df = pd.DataFrame(index=[], columns=cols)
        l_df.to_csv(l_df_dir)

    # ANN.csvにおいて、指定した条件を満たす行だけqueryとして抜き出す
    l_query = l_df.query(learn_condition + '&' + signal_condition + '&' + trans_condition + '&Lmax=='+str(Lmax))

    #if epochsを含む指定された条件を満たす結果がある: 何もしない
    if len(l_query) > 0 and l_query['epochs'].max() >= epochs:
        print('指定された条件の学習結果はすでに存在します')
        return None, None, None
    else:
        # if epochs以外の指定された条件を満たす結果がある: パラメータを読み込む if ない: 新しくモデルを作成する
        if len(l_query) > 0:
            index = l_query['epochs'].idxmax()
            trained_epochs = l_query['epochs'][index]
            model = CVNN(input_dim=sampling*tap, output_dim=1, hidden_neuron=neuron, activation=activation).to(device)
            model.load_state_dict(torch.load(l_query['params_path'][index]))
        else:
            trained_epochs = 0
            model = CVNN(input_dim=sampling*tap, output_dim=1, hidden_neuron=neuron, activation=activation).to(device)
    return model, l_df_dir, trained_epochs

In [19]:
#結果と条件を保存しない
def train_cvnn(model_info, signal_info, tap, PdBm):
    device = torch.device('cpu') #'cuda' if torch.cuda.is_available() else 
    print('Device available now:', device)
    
    lc = model_info['linear_compensation']
    sampling = model_info['sampling']
    #tap = model_info['tap']
    batch_size = model_info['batch_size']
    neuron = model_info['neuron']
    epochs = model_info['epochs']
    lr = model_info['lr']
    activation = model_info['activation']
    form = model_info['form']
    #PdBm = model_info['PdBm']
    Lmax = model_info['Lmax']

    max_tap = 501
    n = 32  # 1シンボルあたりのサンプリング数[/symbol]
    equalize = False  # 各シンボル数を均等にするか
    baudrate = 28  # ボーレート[GBaud]
    Ledfa = 100  # EDFAスパン[km]
    stepedfa = 30  # SSFMの繰り返し計算ステップ数
    gamma = 2.8  # 非線形係数[/W/km]
    D = 16  # 分散パラメータ[ps/nm/km]
    Alpha = 0.16  # 伝送損失[dB/km]
    NF = 4  # ASE雑音指数[dB]
    ase = True  # ASE雑音を考慮するか

    # 指定した学習条件と伝送条件
    trans_condition = 'form=="'+str(form)+'"&n=='+str(n)+'&equalize=='+str(equalize)+'&baudrate=='+str(baudrate)+'&PdBm=='+str(PdBm)+'&Ledfa=='+str(Ledfa)+'&stepedfa=='+str(stepedfa)+'&\
                                    gamma=='+str(gamma)+'&D=='+str(D)+'&Alpha=='+str(Alpha)+'&NF=='+str(NF)+'&ase=='+str(ase)

    x, y, _, _ = load_signal(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax)
    
    # 平均,標準偏差の計算
    mean = np.mean(x)
    std = np.std(x)
    
    model = CVNN(input_dim=sampling*tap, output_dim=1, hidden_neuron=neuron, activation=activation).to(device)

    # dataset, dataloaderの作成
    train_dataset = Dataset(x=x, y=y, mean=mean, std=std)
    train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    dataloaders_dict = {'train': train_dataloader}

    # 損失関数, オプティマイザの作成
    criterion = EVMLoss()
    optimizer = optim.Adam(params=model.parameters(), lr=lr)

    # モデルのトレーニング
    model = train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=epochs)

In [20]:
#結果と条件を../data/params/CVNN.csvに保存する
def train_cvnn_with_save(model_info, signal_info, tap, PdBm):
    device = torch.device('cpu') #'cuda' if torch.cuda.is_available() else 
    print('Device available now:', device)
    
    lc = model_info['linear_compensation']
    sampling = model_info['sampling']
    #tap = model_info['tap']
    batch_size = model_info['batch_size']
    neuron = model_info['neuron']
    epochs = model_info['epochs']
    lr = model_info['lr']
    activation = model_info['activation']
    form = model_info['form']
    #PdBm = model_info['PdBm']
    Lmax = model_info['Lmax']
    
    max_tap = 501
    n = 32  # 1シンボルあたりのサンプリング数[/symbol]
    equalize = False  # 各シンボル数を均等にするか
    baudrate = 28  # ボーレート[GBaud]
    Ledfa = 100  # EDFAスパン[km]
    stepedfa = 30  # SSFMの繰り返し計算ステップ数
    gamma = 2.8  # 非線形係数[/W/km]
    D = 16  # 分散パラメータ[ps/nm/km]
    Alpha = 0.16  # 伝送損失[dB/km]
    NF = 4  # ASE雑音指数[dB]
    ase = True  # ASE雑音を考慮するか

    # 指定した学習条件と伝送条件
    learn_condition = 'linear_compensation=='+str(lc)+'&sampling=='+str(sampling)+'&tap=='+str(tap)+'&max_tap=='+str(max_tap)+'&batch_size=='+str(batch_size)+'&neuron=='+str(neuron)+'&learning_rate=='+str(lr)+'&activation=="'+str(activation)+'"'
    trans_condition = 'form=="'+str(form)+'"&n=='+str(n)+'&equalize=='+str(equalize)+'&baudrate=='+str(baudrate)+'&PdBm=='+str(PdBm)+'&Ledfa=='+str(Ledfa)+'&stepedfa=='+str(stepedfa)+'&\
                                    gamma=='+str(gamma)+'&D=='+str(D)+'&Alpha=='+str(Alpha)+'&NF=='+str(NF)+'&ase=='+str(ase)
    
    x, y, signal_condition, signal_list = load_signal(signal_info, trans_condition, lc, sampling, tap, max_tap, n, Lmax)
    if x is not None:
        model, l_df_dir, trained_epochs = load_model(learn_condition, trans_condition, signal_condition, device, sampling, tap, neuron, epochs, activation, Lmax)
        
        if model is not None:
            train_samples = len(x)
    
            # 平均,標準偏差の計算
            mean = np.mean(x)
            std = np.std(x)
            
            # dataset, dataloaderの作成
            train_dataset = Dataset(x=x, y=y, mean=mean, std=std)
            train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            dataloaders_dict = {'train': train_dataloader}

            # 損失関数, オプティマイザの作成
            criterion = EVMLoss()
            optimizer = optim.Adam(params=model.parameters(), lr=lr)

            # モデルのトレーニング(50epochsずつ学習し、50epochsずつパラメータを保存する)
            for epoch in range(trained_epochs, epochs, 50):
                model = train_model(device=device, model=model, dataloaders_dict=dataloaders_dict, criterion=criterion, optimizer=optimizer, epochs=50, epochs_section=[epoch, epochs])

                # 学習済みパラメータを保存し、CVNN.csvに保存先を記入する
                l_df = pd.read_csv(l_df_dir, index_col=0)
                params_path = '../data/params/CVNN/params_' + str(len(l_df)).zfill(10) + '.pth'
                torch.save(model.state_dict(), params_path)
                sr = pd.Series([lc, sampling, tap, max_tap, batch_size, neuron, epoch + 50, lr, activation] + signal_list + [form, n, equalize, baudrate, PdBm, Ledfa, stepedfa, gamma, D, Alpha, NF, Lmax, ase, params_path, train_samples], index=l_df.columns)
                l_df = l_df.append(sr, ignore_index=True)
                l_df.to_csv(l_df_dir)

In [23]:
model_info = {'linear_compensation': True,
                          'sampling': 4,
                          'batch_size': 100,
                          'neuron': 320,
                          'epochs': 500,
                          'lr': 0.001,
                          'activation': 'CReLU', # 'CReLU' or 'CSigmoid' or 'modReLU'
                          'form': 'RZ16QAM',
                          'Lmax': 2500}

signal_info_random = {'signal_type': 'random',
                                         'seed': '1234',
                                         'bit_num': 50000}

signal_info_image0 = {'signal_type': 'image',
                                     'target_dir': 'train_0',
                                     'step': 60,
                                     'image_number': '0, 1, 2, 3, 4, 5, 6, 7, 8, 9',
                                     'ebtb': True}

signal_info_image1 = {'signal_type': 'image',
                                     'target_dir': 'train_0',
                                     'step': 60,
                                     'image_number': '0, 5, 6, 12, 17, 33, 35, 45, 52, 53',
                                     'ebtb': True}

tap = 25
train_cvnn_with_save(model_info, signal_info_image1, tap, 1)

Device available now: cpu
0:00:06 | Epoch: 1/500 | train Loss: 0.3399 | EVM: 39.63
0:00:06 | Epoch: 2/500 | train Loss: 0.1769 | EVM: 17.74
0:00:06 | Epoch: 3/500 | train Loss: 0.154 | EVM: 15.44
0:00:06 | Epoch: 4/500 | train Loss: 0.1434 | EVM: 14.37
0:00:06 | Epoch: 5/500 | train Loss: 0.1353 | EVM: 13.56
0:00:06 | Epoch: 6/500 | train Loss: 0.1305 | EVM: 13.08
0:00:06 | Epoch: 7/500 | train Loss: 0.1271 | EVM: 12.74
0:00:06 | Epoch: 8/500 | train Loss: 0.1238 | EVM: 12.4
0:00:06 | Epoch: 9/500 | train Loss: 0.1225 | EVM: 12.27
0:00:06 | Epoch: 10/500 | train Loss: 0.1186 | EVM: 11.88
0:00:06 | Epoch: 11/500 | train Loss: 0.1177 | EVM: 11.8
0:00:06 | Epoch: 12/500 | train Loss: 0.1193 | EVM: 11.95
0:00:06 | Epoch: 13/500 | train Loss: 0.1127 | EVM: 11.3
0:00:06 | Epoch: 14/500 | train Loss: 0.1117 | EVM: 11.2
0:00:06 | Epoch: 15/500 | train Loss: 0.1126 | EVM: 11.28
0:00:06 | Epoch: 16/500 | train Loss: 0.1089 | EVM: 10.92
0:00:06 | Epoch: 17/500 | train Loss: 0.1096 | EVM: 10.98
0:

0:00:09 | Epoch: 140/500 | train Loss: 0.06601 | EVM: 6.618
0:00:08 | Epoch: 141/500 | train Loss: 0.06674 | EVM: 6.7
0:00:08 | Epoch: 142/500 | train Loss: 0.06612 | EVM: 6.628
0:00:08 | Epoch: 143/500 | train Loss: 0.06911 | EVM: 6.94
0:00:08 | Epoch: 144/500 | train Loss: 0.06779 | EVM: 6.802
0:00:08 | Epoch: 145/500 | train Loss: 0.06603 | EVM: 6.63
0:00:08 | Epoch: 146/500 | train Loss: 0.06638 | EVM: 6.658
0:00:08 | Epoch: 147/500 | train Loss: 0.06492 | EVM: 6.511
0:00:08 | Epoch: 148/500 | train Loss: 0.06507 | EVM: 6.529
0:00:08 | Epoch: 149/500 | train Loss: 0.06342 | EVM: 6.361
0:00:08 | Epoch: 150/500 | train Loss: 0.06803 | EVM: 6.834
0:00:09 | Epoch: 151/500 | train Loss: 0.06753 | EVM: 6.779
0:00:08 | Epoch: 152/500 | train Loss: 0.06446 | EVM: 6.472
0:00:08 | Epoch: 153/500 | train Loss: 0.06487 | EVM: 6.511
0:00:08 | Epoch: 154/500 | train Loss: 0.06689 | EVM: 6.713
0:00:08 | Epoch: 155/500 | train Loss: 0.06523 | EVM: 6.547
0:00:08 | Epoch: 156/500 | train Loss: 0.066

0:00:08 | Epoch: 277/500 | train Loss: 0.05625 | EVM: 5.647
0:00:08 | Epoch: 278/500 | train Loss: 0.05544 | EVM: 5.567
0:00:08 | Epoch: 279/500 | train Loss: 0.05864 | EVM: 5.889
0:00:08 | Epoch: 280/500 | train Loss: 0.05766 | EVM: 5.788
0:00:09 | Epoch: 281/500 | train Loss: 0.06046 | EVM: 6.072
0:00:08 | Epoch: 282/500 | train Loss: 0.05712 | EVM: 5.733
0:00:08 | Epoch: 283/500 | train Loss: 0.05642 | EVM: 5.67
0:00:08 | Epoch: 284/500 | train Loss: 0.05804 | EVM: 5.833
0:00:08 | Epoch: 285/500 | train Loss: 0.05928 | EVM: 5.955
0:00:08 | Epoch: 286/500 | train Loss: 0.05687 | EVM: 5.71
0:00:08 | Epoch: 287/500 | train Loss: 0.05532 | EVM: 5.569
0:00:08 | Epoch: 288/500 | train Loss: 0.05747 | EVM: 5.787
0:00:08 | Epoch: 289/500 | train Loss: 0.05816 | EVM: 5.841
0:00:08 | Epoch: 290/500 | train Loss: 0.05616 | EVM: 5.639
0:00:08 | Epoch: 291/500 | train Loss: 0.05654 | EVM: 5.678
0:00:08 | Epoch: 292/500 | train Loss: 0.05618 | EVM: 5.651
0:00:08 | Epoch: 293/500 | train Loss: 0.0

0:00:06 | Epoch: 415/500 | train Loss: 0.05018 | EVM: 5.039
0:00:06 | Epoch: 416/500 | train Loss: 0.05166 | EVM: 5.187
0:00:06 | Epoch: 417/500 | train Loss: 0.05665 | EVM: 5.69
0:00:05 | Epoch: 418/500 | train Loss: 0.05468 | EVM: 5.492
0:00:06 | Epoch: 419/500 | train Loss: 0.0543 | EVM: 5.463
0:00:06 | Epoch: 420/500 | train Loss: 0.05201 | EVM: 5.229
0:00:06 | Epoch: 421/500 | train Loss: 0.05211 | EVM: 5.231
0:00:06 | Epoch: 422/500 | train Loss: 0.05577 | EVM: 5.605
0:00:06 | Epoch: 423/500 | train Loss: 0.05461 | EVM: 5.488
0:00:06 | Epoch: 424/500 | train Loss: 0.05532 | EVM: 5.568
0:00:06 | Epoch: 425/500 | train Loss: 0.05281 | EVM: 5.299
0:00:05 | Epoch: 426/500 | train Loss: 0.05344 | EVM: 5.373
0:00:06 | Epoch: 427/500 | train Loss: 0.0541 | EVM: 5.439
0:00:05 | Epoch: 428/500 | train Loss: 0.05254 | EVM: 5.276
0:00:06 | Epoch: 429/500 | train Loss: 0.05517 | EVM: 5.551
0:00:06 | Epoch: 430/500 | train Loss: 0.05405 | EVM: 5.428
0:00:06 | Epoch: 431/500 | train Loss: 0.05